In [170]:
import pandas as pd
import random
import numpy as np

In [171]:
splits = [
    'First, {}. Then, {}.',
    '{}, and {}',
    '{}; {}',
    '{}, and then {}',
    '{}, then {}',
    '{} and then {}',
    '{} before {}',
]

p = [1, 2, 2, 2, 2, 1]
p = [i/sum(p) for i in p]
def create_split(row):
    a = row['A']
    b = row['B']
    if a is None or b is None:
        return a['instruction']
    a = a.strip('. ')
    b = b.strip('. ')
    splits = [
        f'First, {a[0].lower() + a[1:]}. Then, {b[0].lower() + b[1:]}.',
        f'{a[0].upper() + a[1:]}, and {b[0].lower() + b[1:]}.'
        f'{a[0].upper() + a[1:]}; {b[0].lower() + b[1:]}.',
        f'{a[0].upper() + a[1:]}, and then {b[0].lower() + b[1:]}.',
        f'{a[0].upper() + a[1:]}, then {b[0].lower() + b[1:]}.',
        f'{a[0].upper() + a[1:]} and then {b[0].lower() + b[1:]}.',
        f'{a[0].upper() + a[1:]} before {b[0].lower() + b[1:]}.',
    ]
    # randomly choose a split
    return np.random.choice(splits, p=p)

In [172]:
data = pd.read_json('alpaca-cleaned_replaced.jsonl', lines=True)
data

,instruction,input,output,idx
0,"First, identify the type of the sentence ""The ...",The cat sat on the mat.,"Identify the type of the sentence ""The cat sat...",3431
1,List four things that help people stay healthy...,,List four things that help people stay healthy...,7127
2,"First, define the meanings of the words 'subse...",,Define the meanings of the words 'subsequent' ...,7923
3,Provide an example of a situation where the pr...,,"Example of a situation where the proverb ""You ...",17116
4,"Name a popular product manufactured by Apple, ...",,Name a popular product manufactured by Apple:\...,32135
...,...,...,...,...
51755,"First, introduce the protagonist and describe ...",,Short story:\nIn the bustling city of Metropol...,50375
51756,Translate the Spanish word back into English t...,Help,Translate the Spanish word back into English:\...,22300
51757,"First, identify the leadership qualities impli...",He took ownership of the project and worked ha...,Identify the leadership qualities implied by t...,18048
51758,"First, analyze the current limitations of GPT ...",,Analyzing the current limitations of GPT model...,43979


In [180]:
# check if the instruction has structure, "First, A. Then, B"
def check_structure(instruction):
    if instruction.find('First, ') != -1 and instruction.find('Then, ') != -1:
        return True, instruction.split('First, ')[1].split('Then, ')[0], instruction.split('Then, ')[1]
    elif instruction.find(', and then ') != -1:
        return True, instruction.split(', and then ')[0], instruction.split(', and then ')[1]
    elif instruction.find(', and ') != -1:
        return True, instruction.split(', and ')[0], instruction.split(', and ')[1]
    elif instruction.find('; ') != -1:
        return True, instruction.split('; ')[0], instruction.split('; ')[1]
    elif instruction.find(', then ') != -1:
        return True, instruction.split(', then ')[0], instruction.split(', then ')[1]
    else:
        return False, None, None

def drop_instruction(row):
    output = row['output']
    A, B = row['A'], row['B']
    if A is not None:
        A = A.strip('.;:, ')
    if B is not None:
        B = B.strip('.;:, ')

    if "Given the original instruction" in output:
        output = output[:output.find("Given the original instruction")].strip("\n ")

    count = output.count(':\n')
    if (count == 0):
        return output
    
    if count == 1:
        s_output = output.split(':\n')
        if '\n' not in s_output[0]:
            s_output = s_output[1:]
            output = '\n'.join(s_output)
        else:
            # output = s_output[0] + '\n' + '\n'.join(s_output[1].split('\n')[:-1])
            output = '\n'.join(s_output[0].split('\n')[:-1]) + '\n' + s_output[1]
    else:
        s_output = output.split(':\n')
        output = ':\n'.join(s_output[1:])
    # elif count == 2:
    #     s_output = output.split(':\n')
    #     s_output = s_output[1:]
    #     output = '\n'.join(s_output[0].split('\n')[:-1]) + '\n' + s_output[1]
    # elif 3 <= count <= 4:
    #     s_output = output.split(':\n')
    #     s_output = s_output[1:]
    #     output = '\n'.join(s_output[0].split('\n')[:-1]) + ':\n'.join(s_output[1:])

    if (A is not None) and (A.lower() in output.lower()):
        output = output[:output.lower().find(A.lower())] + output[output.lower().find(A.lower())+len(A)+1:]
    # if (B is not None) and (B.lower() in output.lower()):
        # output = output[:output.lower().find(B.lower())-1] + output[output.lower().find(B.lower())+len(B)+1:]
    

    return output
    
data['structure'] = data['instruction'].apply(check_structure, 1)
# split the structure into three columns
data['split'] = data['structure'].apply(lambda x: x[0])
data['A'] = data['structure'].apply(lambda x: x[1])
data['B'] = data['structure'].apply(lambda x: x[2])

data['split'].value_counts()

split
True     45974
False     5786
Name: count, dtype: int64

In [181]:
data['new_instruction'] = data.apply(lambda x: create_split(x) if x['split'] else x['instruction'], 1)
# concat keep_data and left
data['new_output'] = data.apply(drop_instruction, 1)

# drop structure, split, A, B columns and rename new_instruction to instruction
# keep_data.drop(columns=['structure', 'split', 'A', 'B', 'instruction']).rename(columns={'new_instruction': 'instruction'}).to_json('alpaca-cleaned_replaced_keep.jsonl', orient='records', lines=True)
data.drop(columns=['structure', 'split', 'A', 'B', 'instruction', 'output']).rename(columns={'new_instruction': 'instruction', 'new_output': 'output'}).to_json('alpaca-cleaned_replaced_without_a.jsonl', orient='records', lines=True)

In [162]:
data.drop(columns=['structure', 'split', 'A', 'B']).to_json('alpaca-cleaned_replaced_left.jsonl', orient='records', lines=True)

In [128]:
count_output = data['output'].str.count(':\n')

# sample two with 0 count and print their output
for i, row in data[count_output == 3].sample(5).iterrows():
    print(row['instruction'])
    print('Before rewrite:')
    print(row['output'])
    print('After rewrite:')
    print(drop_instruction(row))
    print('---')
    print()

First, analyze the factors contributing to the struggle of balancing school and extracurricular activities. Then, propose a solution to address these challenges.
Before rewrite:
Analyzing the factors contributing to the struggle of balancing school and extracurricular activities:
1. Time Management: High school students often face a demanding schedule with classes, homework, and extracurricular commitments.
2. Prioritization: It can be challenging to prioritize tasks effectively, leading to stress and feelings of being overwhelmed.
3. Lack of Support: Some students may not have adequate support systems in place to help them manage their responsibilities.
4. Burnout: Overcommitting to activities can result in burnout and negatively impact academic performance.
Propose a solution to address these challenges:
To address the struggle of balancing school and extracurricular activities, high school students can implement the following strategies:
1. Create a Detailed Schedule: Develop a deta